In [1]:
#rm -r *

In [2]:
# Uncomment the line below and run this cell to get your data from github into colab (only runnable in colab, not ordinary jupyter notebook):
#! git clone https://github.com/lkriener/music_generation.git && mv music_generation/* . && rm music_generation -r 

In [3]:
# Uncomment line to install requirements
#! pip install -r colab_requirements.txt

# Music generation using note-to-note LSTMs 
In this part, we will try to generate melodies from the same dataset (Bach Chorals) using recurrent neural networks (RNN) with long short term memory units (LSTM) as in the tutorial 08 of the class, based on the two following blog posts:

- http://karpathy.github.io/2015/05/21/rnn-effectiveness/
- https://towardsdatascience.com/writing-like-shakespeare-with-machine-learning-in-pytorch-d77f851d910c

The idea would be to extend the character-to-character LSTM network to midi files. We will also use another representation of the data, namely 'pianorolls', where a single melody is stored in a matrix with the x-axis for time and the y-axis for the pitch. This is one of the most frequent representations used, although it has some limitations. For instance, there is no way to differentiate between a long note (1 half-note) and 4 short-ones (4 eight-notes). However, it brings a considerable advantage compared to the previous used representations as it can store rhythm, depending on the chosen time-step. 

Moreover, each possible note is considered as a distinct element of a vocabulary. We will then use $N$ input nodes, where $N$ is the size of the vocabulary (i.e., the number of distinct notes, including silences). 
Thus, these pianorolls arrays will be converted into one-hot vectors containing the corresponding pitch (or silence, represented by 0) for each time step, and will be fed to the RNN network defined below. 

## Test of pypianoroll library
We will first use the pypianoroll library - https://salu133445.github.io/pypianoroll/ - to convert a midi file into a multitrack object, get the soprano track, transpose it to C major, create a new multitrack object out of it, and finally write it to a new midi file. 

In [4]:

import pygame
from pypianoroll import Multitrack, Track
import pypianoroll
import os
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [5]:
from src.RNN_utils import *

No GPU available, training on CPU; consider making n_epochs very small.


In [6]:
try:
    home_dir
except NameError:
    home_dir = os.getcwd()

os.chdir(home_dir + "/data/raw/bach")  # go to a folder relative to home dir

In [7]:
from src.dataset_utils import TrackDataset, get_dataset_representation_from_tracks
from torch.utils.data import DataLoader

from sklearn.preprocessing import StandardScaler

feature_scaler = StandardScaler()
label_scaler = StandardScaler()

# iterate over all midi files of folder
import glob
import os
import numpy as np

try:
    home_dir
except NameError:
    home_dir = os.getcwd()
    

In [8]:
# get lower and upper bounds 
voice = 0 
all_pianorolls = get_all_pianorolls(voice, home_dir, beat_resolution=4)
global_lower, global_upper, n_notes = get_extremum_pitches([all_pianorolls])
print('Global lower note : '+ str(global_lower))
print('Global upper note : '+ str(global_upper))
print('Number of notes : '+ str(n_notes))

/Users/deperrois/Documents/BERN/ATML/myenv/lib/python3.7/site-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  RuntimeWarning)


Global lower note : 57
Global upper note : 83
Number of notes : 28


In [9]:
print("Pianoroll sequence of soprano :")
print(all_pianorolls[0:100])

Pianoroll sequence of soprano :
[ 0  0  0  0 70 70 70  0 70 70 70 70 68 68 68 68 66 66 66 66 65 65 65 65
 65 65 65 65 63 63 63 63 63 63 63 63 70 70 70  0 70 70 70 70 72 72 72 72
 74 74 74 74 75 75 75 75 75 75 75 75 74 74 74 74 74 74 74 74 75 75 75 75
 78 78 78 78 77 77 77 77 77 77 75  0 75 75 75 75 75 75 75 75 75 75 75 75
 75 75 75 75]


In order to minimize the size of one-hot encode vectors, we restrict the pitches to values next to 0, using the minimum pitch found in the dataset. 

In [10]:
# scale pianoroll to 0 
print("Pianoroll sequence of soprano scaled to 0:")
all_pianorolls_scaled = scale_pianoroll(all_pianorolls, global_lower)
print(all_pianorolls_scaled[0:100])

Pianoroll sequence of soprano scaled to 0:
[ 0  0  0  0 14 14 14  0 14 14 14 14 12 12 12 12 10 10 10 10  9  9  9  9
  9  9  9  9  7  7  7  7  7  7  7  7 14 14 14  0 14 14 14 14 16 16 16 16
 18 18 18 18 19 19 19 19 19 19 19 19 18 18 18 18 18 18 18 18 19 19 19 19
 22 22 22 22 21 21 21 21 21 21 19  0 19 19 19 19 19 19 19 19 19 19 19 19
 19 19 19 19]


## Generation of a single voice

As a first attempt to generate a melody using LSTMs with pianorolls, we will simply try to generate a single voice (soprano, alto, tenor or bass), training the network on the whole midi file dataset concatenated into a long sequence of notes all translated to the same tonality (here, C Major or its relative A minor). The later operation will particularly facilitate learning on such a small dataset as all melodies will be rescaled to the same tonality, thus preventing abrupt changes among melody batches. 

In [11]:
# Check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

No GPU available, training on CPU; consider making n_epochs very small.


### Define a LSTM network as for note-to-note melody generation
We now build a network to implement note-to-note melody generation using LSTMs units as well as drop-out of the output. This network is highly inspired from the Tutorial 08 of the class, by replacing characters by notes. 

## Training Code

For training the model, we declare a function, where we define an optimizer (Adam) and loss (cross entropy). Then, training and validation data are separated and the hidden state of the RNN is initaliazed. Looping over the training melody batches, we use the functions `get_pianoroll_batches` and `one_hot_encode_batch` in order to build batches and feed them to the network input. This function is also adapted to harmonization that will be defined below. 

Every once a while, we generate some loss statistics (training loss and validation loss) to let us know if the model is training correctly.

In [12]:
# Define and print the net
n_hidden=128
n_layers=3

net = NoteRNN(n_notes, n_hidden, n_layers)
print(net)

# Declaring the hyperparameters
batch_size = 32
seq_length = 100
n_epochs = 20 # start smaller if you are just testing initial behavior

# train the model
train(net, all_pianorolls_scaled, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=50)

NoteRNN(
  (lstm): LSTM(28, 128, num_layers=3, batch_first=True, dropout=0.2)
  (dropout): Dropout(p=0.2)
  (fc): Linear(in_features=128, out_features=28, bias=True)
)


KeyboardInterrupt: 

### Test of the network: generation of single voice 
We will now test the trained network to predict potential melodies starting from a short sequence of notes. Here again, the functions `predict` and `sample` are highly inspired from the Tutorial 8, and slightly modified towards pianorolls data processing. 



The `NoteRNN` network outputs scaled pianorolls. We thus need to send back the obtained pitches to the associated voice range, using the function `unscale_pianoroll`. We then store the pianoroll to a track object, and to a multitrack object, which can directly be used to write a midi file. 

In [ ]:
# Generating new melody
start = 72 - global_lower + 1 # starting with a C 
notes = sample(net, 200, prime=[start,start,start+2,start+2])
print("Generated sample :")
print(notes)
normal_pianoroll = unscale_pianoroll(notes, global_lower) # go back to the track range
print("Generated sample in the pitch range :")
print(normal_pianoroll)
# create a one_hot_pianoroll
one_hot_pianoroll = one_hot_encode_pianoroll(normal_pianoroll, 128)*90 # 90 for the velocity
# store it a in a track object
new_track = Track(pianoroll=one_hot_pianoroll, name='new track')
# create a multitrack made of the generated track object
new_multitrack = Multitrack(tracks=[new_track], tempo = 90, beat_resolution=4)
#write to midi file 
pypianoroll.write(new_multitrack, home_dir + "/results/RNN_track")
modified_midi_filename = home_dir + "/results/RNN_track.mid"

In [ ]:
!apt install fluidsynth
!cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2
!pip install midi2audio
from midi2audio import FluidSynth
from IPython.display import Audio



In [ ]:
FluidSynth("font.sf2").midi_to_audio(modified_midi_filename, 'test.wav')
Audio("test.wav")

In [ ]:

pygame.init()
pygame.mixer.music.load(modified_midi_filename)
pygame.mixer.music.play()

In [ ]:
pygame.mixer.music.stop()

## Train for harmonization

We observed above that it is possible to generate a coherent melody using a slightly modified version of the original character-to-character RNN. Here, we will investigate whether such a network can perform more complex task, such as harmonizing a given voice. For instance, from the soprano sequence, we would like to generate the associated alto voice. 

In [13]:
beat_resolution = 2

In [19]:

all_pianorolls_soprano = get_all_pianorolls(0, home_dir, beat_resolution=beat_resolution)
all_pianorolls_alto = get_all_pianorolls(1, home_dir, beat_resolution=beat_resolution)
all_pianorolls_tenor = get_all_pianorolls(2, home_dir, beat_resolution=beat_resolution)
all_pianorolls_bass = get_all_pianorolls(3, home_dir, beat_resolution=beat_resolution)

In [20]:
list_pianorolls = [all_pianorolls_soprano, all_pianorolls_alto, all_pianorolls_tenor, all_pianorolls_bass]
global_lower, global_upper, n_notes = get_extremum_pitches(list_pianorolls)


print('Global lower note : '+ str(global_lower))
print('Global upper note : '+ str(global_upper))
print('Number of notes : '+ str(n_notes))

Global lower note : 35
Global upper note : 83
Number of notes : 50


We now rescale the pianorolls to values next to 0. 

In [21]:
# scale pianoroll to 0 
all_pianorolls_soprano = scale_pianoroll(all_pianorolls_soprano, global_lower)
all_pianorolls_alto = scale_pianoroll(all_pianorolls_alto, global_lower)
all_pianorolls_tenor = scale_pianoroll(all_pianorolls_tenor, global_lower)
all_pianorolls_bass = scale_pianoroll(all_pianorolls_bass, global_lower)


It is now time to train the network to perform harmonization. Setting the mode to `"harmonization"`, we will call the function `get_pianoroll_batches_harmonization` that generates one-hot vectors of both input and target melodies. 

In [22]:
# Define and print the net
n_hidden=512
n_layers=2

net_harmonization_alto = NoteRNN(n_notes, n_hidden, n_layers)
net_harmonization_tenor = NoteRNN(n_notes, n_hidden, n_layers)
net_harmonization_bass = NoteRNN(n_notes, n_hidden, n_layers)



# Declaring the hyperparameters
batch_size = 64
seq_length = 50
n_epochs = 100 # start smaller if you are just testing initial behavior

In [23]:
# train the model
train(net_harmonization_alto, data=all_pianorolls_soprano, data2=all_pianorolls_alto, mode="harmonization", epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.005, print_every=50)

KeyboardInterrupt: 

In [24]:
train(net_harmonization_tenor, data=all_pianorolls_soprano, data2=all_pianorolls_tenor, mode="harmonization", epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.005, print_every=50)

KeyboardInterrupt: 

In [25]:
train(net_harmonization_bass, data=all_pianorolls_soprano, data2=all_pianorolls_bass, mode="harmonization", epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.005, print_every=50)

KeyboardInterrupt: 

In [26]:
 # Generating harmonization

os.chdir(home_dir + "/data/raw/bach")  # go to a folder relative to home dir
# get one voice from the dataset 
#midi_filename = 'bwv368.mid'
midi_filename = 'bwv155.5.mid'
#midi_filename = 'pkgsc_azalea.mid'
soprano_track = get_track(midi_filename, 0, beat_resolution=beat_resolution, transpose=True)
alto_track_real = get_track(midi_filename, 1, beat_resolution=beat_resolution, transpose=True)
tenor_track_real = get_track(midi_filename, 2, beat_resolution=beat_resolution, transpose=True)
bass_track_real = get_track(midi_filename, 3, beat_resolution=beat_resolution, transpose=True)

soprano_pianoroll = scale_pianoroll(flatten_one_hot_pianoroll(soprano_track.pianoroll), global_lower)
alto_pianoroll_real = scale_pianoroll(flatten_one_hot_pianoroll(alto_track_real.pianoroll), global_lower)
tenor_pianoroll_real = scale_pianoroll(flatten_one_hot_pianoroll(tenor_track_real.pianoroll), global_lower)
bass_pianoroll_real = scale_pianoroll(flatten_one_hot_pianoroll(bass_track_real.pianoroll), global_lower)


# now we can predict the second voice 
alto_pianoroll = sample_harmonization(net_harmonization_alto, soprano_pianoroll, prime = alto_pianoroll_real[:4])

tenor_pianoroll = sample_harmonization(net_harmonization_tenor, soprano_pianoroll, prime = tenor_pianoroll_real[:4])
bass_pianoroll = sample_harmonization(net_harmonization_bass, soprano_pianoroll, prime = bass_pianoroll_real[:4])

# go back to the pitch range 
alto_pianoroll = unscale_pianoroll(alto_pianoroll, global_lower)
tenor_pianoroll = unscale_pianoroll(tenor_pianoroll, global_lower)
bass_pianoroll = unscale_pianoroll(bass_pianoroll, global_lower)

# convert to one-hot representation for track object
alto_onehot = one_hot_encode_pianoroll(alto_pianoroll, 128)*90
tenor_onehot = one_hot_encode_pianoroll(tenor_pianoroll, 128)*90
bass_onehot = one_hot_encode_pianoroll(bass_pianoroll, 128)*90

alto_track = Track(pianoroll=alto_onehot, name='alto gernerated track')
tenor_track = Track(pianoroll=tenor_onehot, name='tenor gernerated track')
bass_track = Track(pianoroll=bass_onehot, name='bass gernerated track')
# create a multitrack made of the initial soprano track and the generated alto
new_multitrack = Multitrack(tracks=[soprano_track, alto_track, tenor_track, bass_track], tempo = 90, beat_resolution=beat_resolution)

multitrack_real = Multitrack(tracks=[soprano_track, alto_track_real, tenor_track_real, bass_track_real], tempo = 90, beat_resolution=beat_resolution)
#write to midi file 
pypianoroll.write(new_multitrack, home_dir + "/results/RNN_harmonization_track")
pypianoroll.write(multitrack_real, home_dir + "/results/RNN_harmonization_track_real")

In [27]:
modified_midi_filename = home_dir + "/results/RNN_harmonization_track.mid"
FluidSynth("font.sf2").midi_to_audio(modified_midi_filename, 'test_fake.wav')
Audio("test_fake.wav")

NameError: name 'FluidSynth' is not defined

In [ ]:
modified_midi_filename_real = home_dir + "/results/RNN_harmonization_track_real.mid"
FluidSynth("font.sf2").midi_to_audio(modified_midi_filename_real, 'test_real.wav')
Audio("test_real.wav")

In [28]:
modified_midi_filename = home_dir + "/results/RNN_harmonization_track.mid"
pygame.init()
pygame.mixer.music.load(modified_midi_filename)
pygame.mixer.music.play()

In [29]:
pygame.mixer.music.stop()